In [ ]:
import sys  
sys.path.append('../../Tamil-English-Transliterator')

In [2]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
import re
import emoji
import pandas as pd
import tamil_unicode_map
from nltk.corpus import words
from ta_en_transliterate import TamilTransliterator

In [4]:
data_dir_1 = "../Original Data"
data_dir_2 = "../Trans Data"

In [ ]:
train = pd.read_csv(data_dir_1+'/train.csv')
train.head()

,id,text,label
0,3268,Indha ariya kandupidippin moolam neenga solla ...,0
1,6239,@vijayakumarp7959 unmai therincha nee pesu,0
2,5859,Inga erukka yella dev... boys vadakkan vadakk...,1
3,3519,பீகாரி பிரசாந்த் கிஷோரிடம் கொடுத்த 350 கோடியை ...,1
4,5136,Mumbai Bangalore la 80% percentage outsiders,1


In [6]:
dev = pd.read_csv(data_dir_1+'/dev.csv')
dev.head()

,id,text,label
0,650,"HOSUR also mini North India bro ,,",0
1,2397,@rajaa9979 வணிகர் சங்கங்களின் தலைமை தமிழரிடத...,0
2,1174,Already Telangana becomes Hindi belt 🥺,0
3,5851,வட மாநிலத்தொழிளார் தமிழ்நாட்டினரரை தாக்கினால் ...,0
4,4180,தமிழன் வட இந்தியாவிலும் கேரளா ஆந்திரா கர்ந...,0


In [7]:
test = pd.read_csv(data_dir_1+'/test.csv')
test.head()

,id,text
0,4594,Humanity paatha intha slag countries prblm var...
1,3593,Avanga Holi kondada oruku poranga da
2,2533,Avangala avanga ooruke anupura vazhiya parunga.
3,6975,பைத்தியமாடா நீ.. நான் சோழிய வேளாளர். நான் என் ...
4,6011,Caste is there parayan think to marry higheru...


In [8]:
# def preprocess(text):

#     text = text.replace('\n', ' ')
#     text = emoji.demojize(text, delimiters=(" :", ": "))

#     tamil_pattern = r'[\u0B80-\u0BFF]+'
#     t = TamilTransliterator(tamil_unicode_map.charmap)

#     def replace_tamil(match):
#         return t.transliterate(match.group(0))[1]

#     text = re.sub(tamil_pattern, replace_tamil, text)
#     text = re.sub(r'http[s]?://[a-zA-Z0-9./_-]+|www\.[a-zA-Z0-9./_-]+', '', text)

#     def reduce_match(match):
#         return re.sub(r'(.)\1+', r'\1', match.group(0))

#     text = re.sub(r'\b[a-zA-Z]+\b', reduce_match, text)
#     text = re.sub(r'\s+', ' ', text).strip()

#     return text

In [9]:
# english_words = set(words.words())  

# def preprocess(text):
#     text = text.replace('\n', ' ')
#     text = emoji.demojize(text, delimiters=(" :", ": "))

#     tamil_pattern = r'[\u0B80-\u0BFF]+'
#     t = TamilTransliterator(tamil_unicode_map.charmap)

#     def replace_tamil(match):
#         return t.transliterate(match.group(0))[1]

#     text = re.sub(tamil_pattern, replace_tamil, text)
#     text = re.sub(r'http[s]?://[a-zA-Z0-9./_-]+|www\.[a-zA-Z0-9./_-]+', '', text)

#     def reduce_repeated_letters(word):
#         if re.match(r'^[a-zA-Z]+$', word) and word.lower() not in english_words:
#             return re.sub(r'(.)\1+', r'\1', word)
#         return word

#     text = " ".join([reduce_repeated_letters(word) for word in re.findall(r'\S+', text)])

#     return text.strip()

In [ ]:
english_words = set(words.words())  

def preprocess(text):

    text = text.replace('\n', ' ')
    text = emoji.demojize(text, delimiters=(" :", ": "))

    tamil_pattern = r'[\u0B80-\u0BFF]+'
    t = TamilTransliterator(tamil_unicode_map.charmap)

    def replace_tamil(match):
        return t.transliterate(match.group(0))[1]

    text = re.sub(tamil_pattern, replace_tamil, text)
    text = re.sub(r'http[s]?://[a-zA-Z0-9./_-]+|www\.[a-zA-Z0-9./_-]+', '', text)

    def reduce_repeated_letters(word):
        if word.lower() in english_words:
            return word
        word = re.sub(r'([A-Za-z])\1+', r'\1', word)
        word = re.sub(r'([A-Z])([a-z])\2+', r'\1\2', word) 
        return word

    text = re.sub(r'[A-Za-z]+', lambda m: reduce_repeated_letters(m.group()), text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^\x20-\x7E]', '', text)
    text = text.lower().strip()

    return text

In [11]:
for i in range(len(train['text'])):
    train.loc[i, 'text'] = preprocess(train.loc[i, 'text'])

for i in range(len(dev['text'])):
    dev.loc[i, 'text'] = preprocess(dev.loc[i, 'text'])

for i in range(len(test['text'])):
    test.loc[i, 'text'] = preprocess(test.loc[i, 'text'])

In [12]:
test.head()

,id,text
0,4594,humanity patha intha slag countries prblm varathu
1,3593,avanga holi kondada oruku poranga da
2,2533,avangala avanga oruke anupura vazhiya parunga.
3,6975,paiththiyamada ni.. nan soliya velalar. nan en...
4,6011,caste is there parayan think to marry higherup...


In [13]:
train.to_csv(data_dir_2+'/train_processed.csv', index=False)
dev.to_csv(data_dir_2+'/dev_processed.csv', index=False)
test.to_csv(data_dir_2+'/test_processed.csv', index=False)